Authors: Allard Marc-Antoine

Course: FIN-407

---

## Sentiment Analysis Benchmark

1. Exploration of the training set


2. Models training:
    1. CBOW + Classification Algo
    2. FastText
    3. LSTM/GRU
    4. RoBERTa
    5. FinBERT


---
### Import

In [ ]:
# For Colab
# !pip instal requirements.txt

In [2]:
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import RAdam
import random
from tqdm import tqdm

from datasets import Dataset as HFDataset
from datasets import concatenate_datasets
from torch.utils.data import DataLoader

from transformers import (AutoConfig,
                          AutoTokenizer,
                          AutoModelForSequenceClassification,
                          DataCollatorWithPadding,
                          TrainingArguments,
                          AdamW,
                          get_scheduler,
                          Trainer)

import evaluate
accuracy = evaluate.load("accuracy")

---
### Helper func

---
### Exploration of the training set

---
### Models training

#### 1. CBOW

#### 2. FastText

#### 3. LSTM/GRU

#### 4. RoBERTa

#### 5. FinBERT